### 책소개 / 핵심문장 을 이용한 '생성문장', '해시태그' 생성 프롬프트 및 랭체인

In [25]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
from langchain_upstage import ChatUpstage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
import os
import time
load_dotenv()
API_KEY = os.getenv("API_KEY")

# to generate sentence
sentence_system_template = SystemMessagePromptTemplate.from_template("""
당신은 작가의 문체를 학습하고 독자에게 전달하고자 하는 메시지를 생성하는 전문가입니다.
주어진 책의 소개글과 핵심 문장을 기반으로 **작가의 문체를 반영한 짧은 메시지를 작성**하세요.                                                                  

작성 조건 : 
- 30자 이내로 작성하세요.
- 작가의 이름을 포함하지 마세요.
- 난해한 표현을 피하세요.
- **문어체를 사용하세요.**                                                                                                                                                                                                                                                                                                                                                                                                                        
""")
sentence_user_template = HumanMessagePromptTemplate.from_template("""
아래 [text_description]는 이 책의 소개글 입니다. 제 3자의 시선으로 이 책에 대해 알려주는 글 입니다.
{text_description}

아래 [text_key_sentences]는 이 책의 본문에서 추출한 핵심문장들입니다. 작가가 직접 쓴 문장이므로 문체가 담겨있습니다. 문체를 학습하세요.
{text_key_sentences}

위의 내용을 기반으로, 독자에게 작가가 전달하고 싶은 메시지를 작성해주세요.  
단, 이 작가의 문체를 반영해야 합니다.
**책의 핵심 내용을 요약하여 담고 있어야 합니다.**
""")
sentence_prompt_template = ChatPromptTemplate.from_messages([sentence_system_template, sentence_user_template])



# to generate hashtag
hashtag_system_template = SystemMessagePromptTemplate.from_template("""
당신은 책에 대한 정보를 바탕으로 적절한 해시태그를 생성하는 전문가입니다.
책의 소개글과 핵심 문장들을 분석하여 관련된 해시태그만 10개 작성하세요.
해시태그의 설명은 필요없습니다.                                                                   
""")

hashtag_user_template = HumanMessagePromptTemplate.from_template("""
아래 [text_description]는 이 책의 소개글 입니다. 제 3자의 시선으로 이 책에 대해 알려주는 글 입니다.
{text_description}

아래 [text_key_sentences]는 이 책의 본문에서 추출한 핵심문장들입니다. 작가가 직접 쓴 문장이므로 문체가 담겨있습니다.
{text_key_sentences}

위 내용을 바탕으로 관련된 해시태그 딱 10개만 작성하세요.
- 책의 핵심 주제와 감성을 반영해야 합니다.
- 일반적인 태그 (#독서, #책추천)나 작가이름(#한강에세이), 카테고리(#힐링)보다 책의 개성을 보여주는 태그를 포함하세요.   
- 감성적 태그 (#마음에_와닿는_책), 장르 태그 (#심리치유), 실용적 태그 (#자기계발서) 등을 혼합하세요. 
- 해시태그 앞에는 반드시 `#`을 붙이고, 공백 없이 작성하세요.
- **절대 `_`(언더바)를 사용하지 마세요.** (예: `#책_추천` → `#책추천`으로 수정)
- **출력 시 `_`(언더바)가 포함된 해시태그가 나오면 실패한 결과로 간주합니다.**  
                                                                                                                                                                                             
**❌ 피해야 할 예시**                                                                 
- #일의_감각 (잘못된 형식, `_` 포함)
- #브랜드_스토리 (잘못된 형식, `_` 포함)                                                                

**✅ 올바른 예시**
- #일의감각 (공백 없이 작성)
- #브랜드스토리 (공백 없이 작성)                                                                                                                                                                                                                                                 
""")

hashtag_prompt_template = ChatPromptTemplate.from_messages([hashtag_system_template, hashtag_user_template])



# to generate letter
letter_system_template = SystemMessagePromptTemplate.from_template("""
당신은 이 책의 작가입니다. 이 책을 찾은 독자들에게 책을 추천하는 편지를 씁니다.

편지 작성 기준 :
- **반드시 200자이내**로 작성하세요.(300자를 초과하면 실패한 결과로 간주합니다.)
- 마치 독자와 직접 대화하는 듯한 편지 형식으로 작성하세요.
- 독자가 이 책을 읽고 싶어지도록 **기대감을 주는 내용**이어야 합니다.
- **작가의 인사나, 소개는 꼭 작성하지 않습니다.**                                                                                                                                                                                                                                                                                                                                            
""")

letter_user_template = HumanMessagePromptTemplate.from_template("""
아래 [text_description]는 이 책의 소개글 입니다. 제 3자의 시선으로 이 책에 대해 알려주는 글 입니다.
{text_description}

아래 [text_key_sentences]는 이 책의 본문에서 추출한 핵심문장들입니다. 작가가 직접 쓴 문장이므로 문체가 담겨있습니다.
{text_key_sentences}

위 내용과 문체를 학습한 후, 이 책의 작가로서 독자에게 강렬한 편지를 남겨주세요.
- 책을 읽기 전 독자가 책을 기대할 수 있게 감정을 전달하세요.
- 책의 **분위기와 작가 특유의 문체를 자연스럽게 반영**하세요.
- **꼭 독자에게 직접 말하는 듯한 어조를 사용하세요.**
- **반드시 200자이내**로 작성하세요.(300자를 초과하면 실패한 결과로 간주합니다.)
- **작가의 인사나, 소개는 꼭 작성하지 않습니다.** 
                                                                
**❌ 피해야 할 예시** 
- 안녕하세요, 독자님.
- 안녕하세요, 독자에게.
- 안녕하세요                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
""")

letter_prompt_template = ChatPromptTemplate.from_messages([letter_system_template, letter_user_template])




# 2. Initialize the LLM (using OpenAI GPT as an example)
llm = ChatUpstage(api_key=API_KEY, model_name="solar-pro", temperature=0.7)

# 3. Create the LangChain using the template and LLM
sentence_chain = LLMChain(llm=llm, prompt=sentence_prompt_template)
hashtag_chain = LLMChain(llm=llm, prompt=hashtag_prompt_template)
letter_chain = LLMChain(llm=llm, prompt=letter_prompt_template)

In [26]:
# 4. Function to process the text and get the result
def get_author_message(text_description, text_key_sentences):
    time.sleep(1)
    return sentence_chain.run({
            "text_description": text_description,
            "text_key_sentences": text_key_sentences
        }
    )

def get_hashtags(text_description, text_key_sentences):
    time.sleep(1)
    return hashtag_chain.run({
            "text_description": text_description,
            "text_key_sentences": text_key_sentences
        }
    )

def get_letter(text_description, text_key_sentences):
    time.sleep(1)
    return letter_chain.run({
            "text_description": text_description,
            "text_key_sentences": text_key_sentences
        }
    )





# 5. Function to process multiple books
def process_multiple_books(text_description, text_key_sentences, isbn_list):
    results = {}
    for idx, (text_ds, text_ks) in enumerate(zip(text_description, text_key_sentences)):
        print(f"Processing book {idx}...")
        message = get_author_message(text_ds, text_ks)
        hashtags = get_hashtags(text_ds, text_ks)
        letter = get_letter(text_ds, text_ks)
        results[f"Book {idx}"] = {
            "message": message,
            "hashtags": hashtags,
            "letter": letter,
            "isbn": isbn_list[idx]
        }

        # 파일 저장 경로 설정
        file_path = "notebook/data"
        os.makedirs(file_path, exist_ok=True)  # 디렉토리가 없으면 생성

        # JSON 파일로 저장
        output_file = os.path.join(file_path, "llm_output.json")
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)

        print(f"Results have been saved to {output_file}")

    return results

### 책 데이터로 부터 리스트 생성

In [28]:
import json
import re

# JSON 데이터 로드
with open('../data/scraping/filtered_book_unique.json', "r", encoding="utf-8") as file:
    data = json.load(file)

print(len(data))
# description과 key_sentences를 통합한 리스트 생성
description_list = []
key_sentences_list = []
isbn_list = []
isbn_loss_idx = []

for idx, el in enumerate(data[0:1001]):
    # description과 key_sentences를 가져옴
    description = el.get('description', '')  # description이 없으면 빈 문자열
    key_sentences = el.get('key_sentences',)  # key_sentences가 없으면 빈 문자열
    isbn_str = str(el.get('isbn', ''))  # isbn13을 문자열로 변환

    # 숫자만 추출하고 빈 문자열인 경우 처리
    isbn_numeric = re.sub(r"\D", "", isbn_str)
    if isbn_numeric:  # 숫자가 있는 경우에만 int 변환
        isbn13 = int(isbn_numeric)
    else:
        isbn13 = None
        isbn_loss_idx.append(idx)


    # 리스트에 추가
    description_list.append(description)
    key_sentences_list.append(key_sentences)
    isbn_list.append(isbn13)

# 결과 확인
print(f"총 {len(key_sentences_list)}개의 항목이 생성되었습니다.")
print(description_list[:1])
print(key_sentences_list[:1])  # 샘플 출력

1781
총 1001개의 항목이 생성되었습니다.
['행복을 찾는 방법이 아니라불행에 대한 수비력을 길러주는58가지 인생 이야기《1cm 다이빙》, 《홈 in 홈》으로 많은 사랑을 받았던 에세이스트 태수가 2년 만의 신작 《어른의 행복은 조용하다》로 돌아왔다. 이번 신작에서 저자 태수는 그동안 선보였던 이야기보다 한층 성숙하고 현명하게 삶의 행복에 가까워지는 방법에 대해 이야기한다. 새로운 것, 짜릿한 것, 남들보다 높은 곳에서 행복하고 싶어 발버둥치는 사람들에게 행복은 꼭 그런 데에만 있는 게 아니라 불행해지지 않는 것에서부터 출발한다고 조용히 일러준다.《어른의 행복은 조용하다》는 요란한 세상에서도 흔들리지 않고 내 삶을 살아가는 튼튼하고 단단한 태도를 담아냈다. 시끌벅적 기쁜 일을 찾아다니기보다도, 울 일이 없고 별다른 나쁜 일이 없는 하루를 만들어야 한다고 강조한다. 그랬을 때에야 비로소 진정한 행복이 우리 곁에 온다는 사실을 다시 한번 일깨워준다.이 책을 먼저 읽은 독자들의 반응 역시 뜨겁다. “울고 싶어지는 날이면 태수 작가의 글을 찾는다. 충분히 울고 다시 나아가기 위해”, “『불편한 편의점』 이후로 오랜만에 끝나지 않길 바라던 책”이라며 극찬하하며, 저자의 SNS에서 5만여 명의 독자에게 선보인 선공개 원고에도 빨리 책으로 출간해달라는 요청이 쇄도했다.이제 당신 차례다. 《어른의 행복은 조용하다》를 읽고 현명하게 행복을 찾아보자. 혹자는 여전히 의심스러운 목소리로 그토록 조용한 인생에서도 행복을 발견할 수 있냐고 묻지만 저자는 확신에 찬 목소리로 단호하게 답한다. “물론”이라고.']
['그냥 지금처럼 살아라. 그렇게 살되 어떤 감정조차 책임질 수 없을 만큼 힘든 날, 마음속이 온통 타인의 감정으로 가득해 당장이라도 터져버릴 것 같은 그런 날. 부러 나밖에 없는 공간으로 도망가자. 그 조용한 공간에서 자신에게도 이렇게 말할 기회를 주자.“나 안 괜찮아.” 가끔은 남에게 줬던 섬세함을 나에게도 허락하자.포기가 습관이 되면 포기하지 않아도 되는 것까지 포기

In [29]:
print(len(data))
print(len(description_list))
print(len(key_sentences_list))
print(len(isbn_list))
print(isbn_list[0])
print(len(isbn_loss_idx))
# print(data[4884])

1781
1001
1001
1001
9791169851053
0


### 예외처리 및 실행

- 1시간당 750개 처리

In [140]:

# Example usage
if __name__ == "__main__":

    if len(description_list) != len(data) or len(key_sentences_list) != len(data):
        print("Warning: The length of text_description and text_key_sentences is not the same.")
        min_length = min(len(description_list), len(key_sentences_list))
        print(f"Processing only the first {min_length} items.")
    else:
        min_length = len(description_list)


    try:
        text_description = description_list[:min_length]
        text_key_sentences = key_sentences_list[:min_length]
    except Exception as e:
        print(f"An error occurred while processing the books: {e}")

    results = process_multiple_books(text_description, text_key_sentences, isbn_list)

    output_filename = "book_information.json"  # 저장할 파일 이름
    with open(output_filename, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

    print(f"✅ JSON 파일 저장 완료: {output_filename}")

    
    for book, data in results.items():
        print(f"{book} 메시지: {data['message']}")
        print(f"{book} 해시태그: {data['hashtags']}")
        print(f"{book} 작가의편지: {data['letter']}",'\n')

Processing only the first 780 items.
Processing book 0...
Results have been saved to notebook/data\llm_output.json
Processing book 1...
Results have been saved to notebook/data\llm_output.json
Processing book 2...
Results have been saved to notebook/data\llm_output.json
Processing book 3...
Results have been saved to notebook/data\llm_output.json
Processing book 4...
Results have been saved to notebook/data\llm_output.json
Processing book 5...
Results have been saved to notebook/data\llm_output.json
Processing book 6...
Results have been saved to notebook/data\llm_output.json
Processing book 7...
Results have been saved to notebook/data\llm_output.json
Processing book 8...
Results have been saved to notebook/data\llm_output.json
Processing book 9...
Results have been saved to notebook/data\llm_output.json
Processing book 10...
Results have been saved to notebook/data\llm_output.json
Processing book 11...
Results have been saved to notebook/data\llm_output.json
Processing book 12...
Res

RateLimitError: Error code: 429 - {'error': {'message': 'You have reached API request limit. Please wait and try again later. If your use-case require a higher rate limit, please request at https://support.upstage.ai', 'type': 'too_many_requests', 'param': '', 'code': 'too_many_requests'}}